### **Residual Block**

In [2]:
from torch import nn

class ResidualBlock(nn.Module):
    def __init__(self, block):
        super().__init__()
        self.block = block
    def forward(self, x):
        return self.block(x) + x # Skip Connection

---
### **ResNet**

In [8]:
class Conv6Res(nn.Module):
    def __init__(self, n_class=10):
        super().__init__()
        self.name = 'conv6res'
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1), # 입력채널(RGB), 출력채널, 커널 크기(3x3), Stride, Padding
            nn.BatchNorm2d(32), # 배치 정규화(입력 데이터의 분포를 정규화함)
            nn.ReLU(), # 활성화함수 ReLU로 비선형성 부여

            ResidualBlock(
                nn.Sequential(
                    nn.Conv2d(32, 32, 3, 1, 1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),

                    nn.Conv2d(32, 32, 3, 1, 1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                )
            ),

            ResidualBlock(
                nn.Sequential(
                    nn.Conv2d(32, 32, 3, 1, 1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),

                    nn.Conv2d(32, 32, 3, 1, 1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),

                    nn.Conv2d(32, 32, 3, 1, 1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                )
            ),

            nn.Flatten(),
            nn.Linear(32*32*32, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

### **Training**

In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

# Downloading CIFAR-10 Dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
cifar_tr = datasets.CIFAR10(root=os.getcwd(), train=True, download=True, transform=transform)
cifar_test = datasets.CIFAR10(root=os.getcwd(), train=False, download=True, transform=transform)

# Split training data into train set and validation set
def split_train_valid(dataset, valid_ratio=0.1):
    data_size = len(dataset)
    indices = list(range(data_size))
    np.random.seed(1)
    np.random.shuffle(indices)

    split_point = int(np.floor(valid_ratio*data_size))
    val_index, train_index = indices[:split_point-1], indices[split_point:]

    train = torch.utils.data.Subset(dataset, train_index)
    valid = torch.utils.data.Subset(dataset, val_index)

    return train, valid

cifar_train, cifar_valid = split_train_valid(dataset=cifar_tr)

# Make DataLoaders for train/validation/test datasets
cifar_loaders = [DataLoader(dataset=d, batch_size=128, shuffle=True, drop_last=True) for d in [cifar_train, cifar_valid, cifar_test]]

# Define model to train
model = Conv6Res()
model.to("cuda")

# Define loss function and optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Use GPU if available (cuda if not)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
num_epochs = 40
history = {"train_loss": [], "train_acc": [], "valid_loss":[], "valid_acc":[]} # record of loss and accuracy in each epoch for plotting later
for epoch in range(num_epochs):
    train_loss, train_acc = 0, 0
    model.train()
    for (x, y) in cifar_loaders[0]: # cifar_loaders[0] is train set DataLoader
        x = x.to(device)
        y = y.to(device)

        hypothesis = model(x)
        cost = loss(hypothesis, y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        train_loss += cost.to(device).item()
        train_acc += (hypothesis.argmax(1)==y).type(torch.float).to(device).mean().item()

    train_loss /= len(cifar_loaders[0]) # len(DataLoader) is batch size (ie, 128)
    train_acc /= len(cifar_loaders[0])
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)

    # Evaluate model on validation set
    valid_loss, valid_acc = 0, 0
    model.eval()
    with torch.no_grad():
        for (x, y) in cifar_loaders[1]: # Validation set DataLoader
            x = x.to(device)
            y = y.to(device)

            hypothesis = model(x)
            cost = loss(hypothesis, y)

            valid_loss += cost.to(device).item()
            valid_acc += (hypothesis.argmax(1)==y).type(torch.float).to(device).mean().item()
    
    valid_loss /= len(cifar_loaders[1]) # len(DataLoader) is batch size
    valid_acc /= len(cifar_loaders[1])
    history["valid_loss"].append(valid_loss)
    history["valid_acc"].append(valid_acc)

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, train loss: {train_loss:>6f}, train acc: {train_acc:>3f}, valid loss: {valid_loss:>6f}, valid acc: {valid_acc:>3f}")

# Test the model on the test set
print("===== Test Start =====")
test_loss, test_acc = 0, 0
model.eval()
with torch.no_grad():
    for (x, y) in cifar_loaders[2]:
        x = x.to(device)
        y = y.to(device)

        hypothesis = model(x)
        cost = loss(hypothesis, y)

        test_loss += cost.to(device).item()
        test_acc += (hypothesis.argmax(1)==y).type(torch.float).to(device).mean().item()

test_loss /= len(cifar_loaders[2])
test_acc /= len(cifar_loaders[2])
print(f"Test loss: {test_loss:>6f}, Test acc: {test_acc:>6f}")

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, train loss: 1.744462, train acc: 0.377404, valid loss: 1.511918, valid acc: 0.455329
Epoch: 10, train loss: 0.467130, train acc: 0.835292, valid loss: 1.190502, valid acc: 0.655248
Epoch: 20, train loss: 0.172632, train acc: 0.941440, valid loss: 2.087788, valid acc: 0.647636
Epoch: 30, train loss: 0.099709, train acc: 0.967504, valid loss: 2.735033, valid acc: 0.641426
===== Test Start =====
Test loss: 3.114580, Test acc: 0.641827
